# Phylogenetic Inference using PNUTS

This notebook gives an example on how to do phylogenetic inference using the PNUTS algorithm described in Wahle (2021) ([bioRxiv Paper](https://doi.org/10.1101/2021.03.16.435623))

First the `MCPhylo` package and the `Random` package are loaded.

In [1]:
using MCPhylo;
using Random;
Random.seed!(1234);

The next step is to load the data.

In [2]:
tree, data = make_tree_with_data("Example.nex");

┌ Warning: Keyword datatype not understood, will be ignored
└ @ MCPhylo C:\Users\Johannes\.julia\packages\MCPhylo\wj70o\src\Parser\ParseNexus.jl:89
┌ Warning: Keyword interleave not understood, will be ignored
└ @ MCPhylo C:\Users\Johannes\.julia\packages\MCPhylo\wj70o\src\Parser\ParseNexus.jl:89


The `tree` object contains a r random tree over the leaves specified in the nexus file. You can view a newick string representing the tree by calling the `newick` function on the object.

In [3]:
newick(tree)

"(Lang3:0.9646697763820897,Lang2:0.7899036826169576,((Lang5:0.2986142783434118,Lang4:0.24683718661000897)6:0.646690981531646,Lang1:0.11248587118714015)7:0.9457754052519123)8:1.0;"

The input data needs to be stored in a dictionary to make it accessible to the sampler.

In [4]:
data_dictionary = Dict{Symbol, Any}(
  :data => data
);

Define a model by specifing a prior distribution on the equilibrium frequencis, a Dirichlet prior in this case, and a prior on the phylogenetic tree. In this example the Compound Dirichlet distribution (Zhang, Rannala and Yang 2012. ([paper](https://doi.org/10.1093/sysbio/sys030))) is chosen.

The distribution associated with the data is the `PhyloDist`. It is a distribution whose likelihood function is calculated according to Felsensteins Pruning algorithm ([paper](https://doi.org/10.1007/BF01734359)).

The *Restriction Site Model* for character evolution and no rate variation accross sites is chosen.

In [5]:
model =  Model(
    data = Stochastic(3, (tree, eq_freq) ->  PhyloDist(tree, eq_freq, [1.0], [1.0], Restriction), false, false),
    eq_freq = Stochastic(1, () -> Dirichlet(2,1)),
    tree = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true)
     )

Object of type "Model"
-------------------------------------------------------------------------------
eq_freq:
Object of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
tree:
Object of type "String"
Tree with root:
"no_name"
Length:
0
Height:
0
Number of leave nodes:
0
-------------------------------------------------------------------------------
data:
Object of type "0×0×0 ArrayStochastic{3}"
Array{Float64}(undef,0,0,0)


Select the PNUTS sampler for the phylogenetic tree and the Slice sampler for the equilibrium frequency.

In [6]:
scheme = [PNUTS(:tree, target=0.7, targetNNI=4),
          SliceSimplex(:eq_freq),
          ]
setsamplers!(model, scheme);

Set initial values.

In [7]:
inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree => tree,
    :eq_freq=> rand(Dirichlet(2,1)),
    :data => data_dictionary[:data]
    ),
    ];

Run the MCMC.

In [8]:
sim = mcmc(model, data_dictionary, inits, 5000, burnin=2500,thin=5, chains=1, trees=true)

MCMC Simulation of 5000 Iterations x 1 Chain...

Chain 1:   0% [4:57:59 of 4:58:35 remaining]
Chain 1:  10% [0:05:29 of 0:06:06 remaining]
Chain 1:  20% [0:02:29 of 0:03:06 remaining]
Chain 1:  30% [0:01:28 of 0:02:06 remaining]
Chain 1:  40% [0:00:58 of 0:01:36 remaining]
Chain 1:  50% [0:00:39 of 0:01:18 remaining]
Chain 1:  60% [0:00:27 of 0:01:06 remaining]
Chain 1:  70% [0:00:17 of 0:00:58 remaining]
Chain 1:  80% [0:00:10 of 0:00:51 remaining]
Chain 1:  90% [0:00:05 of 0:00:46 remaining]
Chain 1: 100% [0:00:00 of 0:00:42 remaining]



Object of type "ModelChains"

Iterations = 2505:5000
Thinning interval = 5
Chains = 1
Samples per chain = 500

[2.969713757848305 8.509450311804892 … 0.2853148152901489 -30.128247746966053; 2.969713757848305 8.509450311804892 … 0.34596003243389967 -29.69815663586149; … ; 2.969713757848305 8.509450311804892 … 0.45050639689763894 -30.679143172543817; 2.969713757848305 8.509450311804892 … 0.24984550381883858 -30.80304100333875]